# **Trabalhando** com dask (pela primeira vez)

- https://docs.dask.org/en/latest/dataframe-parquet.html
- https://www.analyticsvidhya.com/blog/2018/08/dask-big-datasets-machine_learning-python/
- https://www.youtube.com/watch?v=WLBEzlwvCp4)

In [ ]:
!pip install "dask[complete]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00


In [ ]:
import dask
import dask.dataframe as dd

import numpy as np
import pandas as pd

In [ ]:
path = '/content/PARTICIPANTES_2024.csv'

In [ ]:
# assume_missing=True, permite que o Dask lide corretamente com dados contendo valores faltantes em colunas que, de outra forma, seriam considerados exclusivamente numéricos.
#df = dd.read_csv(path, assume_missing = True, sep = ';')
df = dd.read_csv(path, encoding='latin1', sep=';')
df.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ENSINO,...,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023
0,210062064233,2024,5,F,1,1,1,1,3,NaN,...,A,B,B,B,D,A,B,B,E,A
1,210062064234,2024,11,F,1,1,1,1,10,NaN,...,A,A,A,A,B,A,B,A,C,A
2,210062064235,2024,11,F,1,1,1,1,9,NaN,...,A,B,B,B,D,B,B,A,D,A
3,210062064236,2024,3,F,1,3,1,2,0,1.0,...,A,B,A,A,A,A,B,A,D,A
4,210062064237,2024,16,M,3,1,1,1,18,NaN,...,A,A,A,A,B,A,B,A,B,A


In [ ]:
print('Quantidade de linhas: ', len(df))
print('Quantidade de colunas: ', len(df.columns)) #df.shape[0].compute()
#df.info() #mostra quantas colunas existem, qual é a primeira e qual é a última, além do count dos tipos dessas colunas

Quantidade de linhas:  4332944
Quantidade de colunas:  38


In [ ]:
print('As colunas presentes no dataFrame são:')
list(df.columns)

As colunas presentes no dataFrame são:


['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'SG_UF_PROVA',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009',
 'Q010',
 'Q011',
 'Q012',
 'Q013',
 'Q014',
 'Q015',
 'Q016',
 'Q017',
 'Q018',
 'Q019',
 'Q020',
 'Q021',
 'Q022',
 'Q023']

### Seleção das variáveis

In [ ]:
#filtro de variáveis
df_subset = df[['NU_INSCRICAO','NU_ANO','TP_FAIXA_ETARIA','TP_SEXO','TP_ESTADO_CIVIL','TP_COR_RACA','TP_NACIONALIDADE','TP_ST_CONCLUSAO','TP_ANO_CONCLUIU','TP_ENSINO','Q001','Q002','Q005','Q006','Q007','Q008','Q011','Q013','Q014','Q015','Q017','Q018','Q020','Q021','Q023']]

In [ ]:
print('Quantidade de linhas: ', len(df_subset))
print('Quantidade de colunas: ', len(df_subset.columns)) #df_subset.shape[0].compute()

Quantidade de linhas:  4332944
Quantidade de colunas:  25


In [ ]:
# Renomeando vairáveis
df_renomeado = df_subset.rename(columns={'TP_FAIXA_ETARIA': 'IDADE',
                                  'Q001': 'ESC_PAI',
                                  'Q002': 'ESC_MAE',
                                  'Q005': 'QTD_PESSOAS_CASA',
                                  'Q006': 'POSSUI_RENDA',
                                  'Q007': 'RENDA_FAM',
                                  'Q008': 'EMPREGADA',
                                  'Q011': 'CARRO',
                                  'Q013': 'GELADEIRA',
                                  'Q014': 'FREEZER',
                                  'Q015': 'MQ_LAVAR_ROUPA',
                                  'Q017': 'ASPIRADOR_PO',
                                  'Q018': 'TELEVISAO',
                                  'Q020': 'WI_FI',
                                  'Q021': 'PC',
                                  'Q023': 'TP_ESCOLA_EM'})

In [ ]:
# Renomeando o conteúdo
mapeamentos = {
    'IDADE': {          # coluna → {código: rótulo}
        '1': 'Menos de 17 anos',
        '2': '17 anos',
        '3': '18 anos',
        '4': '19 anos',
        '5': '20 anos',
        '6': '21 anos',
        '7': '22 anos',
        '8': '23 anos',
        '9': '24 anos',
        '10': '25 anos',
        '11': 'Entre 26 e 30 anos',
        '12': 'Entre 31 e 35 anos',
        '13': 'Entre 36 e 40 anos',
        '14': 'Entre 41 e 45 anos',
        '15': 'Entre 46 e 50 anos',
        '16': 'Entre 51 e 55 anos',
        '17': 'Entre 56 e 60 anos',
        '18': 'Entre 61 e 65 anos',
        '19': 'Entre 66 e 70 anos',
        '20': 'Maior de 70 anos'
    },
    'TP_SEXO': {
        'F': 'Feminino',
        'M': 'Masculino'
    },
      'TP_ESTADO_CIVIL': {
        '0': 'Não informado',
        '1': 'Solteiro(a)',
        '2': 'Casado(a)/Mora com companheiro(a)',
        '3': 'Divorciado(a)',
        '4': 'Viúvo(a)'
    },
      'TP_COR_RACA': {
        'A': 'Branco',
        'B': 'Pardo',
        'C': 'Negro',
        'D': 'Amarelo',
        'E': 'Indigena'
    },
    'TP_NACIONALIDADE': {
        '0': 'Não informado',
        '1': 'Brasileiro(a)',
        '2': 'Brasileiro(a) Naturalizado(a)',
        '3': 'Estrangeiro(a)',
        '4': 'Brasileiro(a) Nato, nascido no exterior'
    },
    'TP_ST_CONCLUSAO' : {
        '1': 'Já concluí o Ensino Médio',
        '2': 'Estou cursando e concluirei o Ensino Médio em 2024',
        '3': 'Estou cursando e concluirei o Ensino Médio após 2024',
        '4': 'Não concluí e não estou cursando o Ensino Médio'
    },
    'TP_ANO_CONCLUIU': {
        '0': 'Não informado',
        '1': '2023',
        '2': '2022',
        '3': '2021',
        '4': '2020',
        '5': '2019',
        '6': '2018',
        '7': '2017',
        '8': '2016',
        '9': '2015',
        '10': '2014',
        '11': '2013',
        '12': '2012',
        '13': '2011',
        '14': '2010',
        '15': '2009',
        '16': '2008',
        '17': '2007',
        '18': 'Antes de 2007'
    },
    'ESC_PAI' : {
        'A': 'Nunca estudou',
        'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental',
        'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental',
        'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.',
        'E': 'Completou o Ensino Médio, mas não completou a Faculdade',
        'F': 'Completou a Faculdade, mas não completou a Pós-graduação',
        'G': 'Completou a Pós-graduação.',
        'H': 'Não sei'
    },
    'ESC_MAE' : {
        'A': 'Nunca estudou',
        'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental',
        'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental',
        'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.',
        'E': 'Completou o Ensino Médio, mas não completou a Faculdade',
        'F': 'Completou a Faculdade, mas não completou a Pós-graduação',
        'G': 'Completou a Pós-graduação.',
        'H': 'Não sei'
    },
    'QTD_PESSOAS_CASA': {
        '1': '1, moro sozinho',
        '2': '2',
        '3': '3',
        '4': '4',
        '5': '5',
        '6': '6',
        '7': '7',
        '8': '8',
        '9': '9',
        '10': '10',
        '11': '11',
        '12': '12',
        '13': '13',
        '14': '14',
        '15': '15',
        '16': '16',
        '17': '17',
        '18': '18',
        '19': '19',
        '20': '20'
    },
    'POSSUI_RENDA' : {
        'A': 'Sim',
        'B': 'Não'
    },
    'RENDA_FAM' : {
        'A':'Nenhuma Renda',
        'B':'Até R$ 1.412,00',
        'C':'De R$ 1.412,01 até R$ 2.118,00',
        'D':'De R$ 2.118,01 até R$ 2.824,00',
        'E':'De R$ 2.824,01 até R$ 3.530,00',
        'F':'De R$ 3.530,01 até R$ 4.236,00',
        'G':'De R$ 4.236,01 até R$ 5.648,00',
        'H':'De R$ 5.648,01 até R$ 7.060,00',
        'I':'De R$ 7.060,01 até R$ 8.472,00',
        'J':'De R$ 8.472,01 até R$9.884,00',
        'K':'De R$ 9.884,01 até R$ 11.296,00',
        'L':'De R$ 11.296,01 até R$ 12.708,00',
        'M':'De R$ 12.708,01 até R$ 14.120,00',
        'N':'De R$ 14.120,01 até R$ 16.944,00',
        'O':'De R$ 16.944,01 até R$ 21.180,00',
        'P':'De R$ 21.180,01 até R$ 28.240,00',
        'Q':'Acima de R$ 28.240,00'
    },
    'EMPREGADA' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
    'CARRO' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
    'GELADEIRA' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
    'FREEZER' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
     'MAQ_LAVAR_ROUPA' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
    'ASPIRADOR_PO' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
    'TELEVISAO' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
    'WI_FI' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
     'PC' : {
        'A': 'Não',
        'B': 'Sim, um ou dois dias por semana',
        'C': 'Sim, três ou quatro dias por semana',
        'D': 'Sim, pelo menos cinco dias por semana'
    },
    'TP_ESCOLA_EM' : {
        'A': 'Somente em escola pública',
        'B': 'Parte em escola pública e parte em escola privada sem bolsa de estudo integral',
        'C': 'Parte em escola pública e parte em escola privada com bolsa de estudo integral',
        'D': 'Somente em escola privada sem bolsa de estudo integral',
        'E': 'Somente em escola privada com bolsa de estudo integral',
        'F': 'Não frequentei escola de Ensino Médio'
    }
}

In [ ]:
tipos = {
    'NU_ANO'            : 'int64', # numérico inteiro
    'IDADE'             : 'category',        # mantém dicionário‑encoding eficiente
    'TP_SEXO'           : 'category',
    'TP_ESTADO_CIVIL'   : 'category',
    'TP_COR_RACA'       : 'category',
    'TP_NACIONALIDADE'  : 'category',
    'TP_ST_CONCLUSAO'   : 'category',
    'TP_ANO_CONCLUIU'   : 'category',
    'TP_ENSINO'         : 'category',
    'ESC_PAI'           : 'category',
    'ESC_MAE'           : 'category',
    'QTD_PESSOAS_CASA'  : 'category',
    'POSSUI_RENDA'      : 'category',
    'RENDA_FAM'         : 'category',
    'EMPREGADA'         : 'category',
    'CARRO'             : 'category',
    'GELADEIRA'         : 'category',
    'FREEZER'           : 'category',
    'MQ_LAVAR_ROUPA'    : 'category',
    'ASPIRADOR_PO'      : 'category',
    'TELEVISAO'         : 'category',
    'WI_FI'             : 'category',
    'PC'                : 'category',
    'TP_ESCOLA_EM'      : 'category'
}

In [ ]:
df_padronizado = df_renomeado.replace(mapeamentos)

In [ ]:
# filtra os tipos para remover as colunas que são datas (cujo tipo é 'datetime64[ns]' ou 'datetime64[ns, UTC])'.
# cria um novo dicionário chamado numeros_e_cats com apenas os tipos numéricos e categóricos
numeros_e_cats = {k: v for k, v in tipos.items()
                  if v not in ('datetime64[ns]', 'datetime64[ns, UTC]')}

# Aplicar conversão de tipo em lote
df_padronizado = df_padronizado.astype(numeros_e_cats)

In [ ]:
print(df_padronizado.dtypes)

NU_INSCRICAO           int64
NU_ANO                 int64
IDADE               category
TP_SEXO             category
TP_ESTADO_CIVIL     category
TP_COR_RACA         category
TP_NACIONALIDADE    category
TP_ST_CONCLUSAO     category
TP_ANO_CONCLUIU     category
TP_ENSINO           category
ESC_PAI             category
ESC_MAE             category
QTD_PESSOAS_CASA    category
POSSUI_RENDA        category
RENDA_FAM           category
EMPREGADA           category
CARRO               category
GELADEIRA           category
FREEZER             category
MQ_LAVAR_ROUPA      category
ASPIRADOR_PO        category
TELEVISAO           category
WI_FI               category
PC                  category
TP_ESCOLA_EM        category
dtype: object


In [ ]:
for col in df_padronizado.columns:
    resultado = df_padronizado.groupby(col).size().compute()
    print(resultado)

NU_INSCRICAO
210062064233    1
210062064234    1
210062064235    1
210062064236    1
210062064237    1
               ..
210066495575    1
210066495576    1
210066495577    1
210066495578    1
210066495579    1
Length: 4332944, dtype: int64
NU_ANO
2024    4332944
dtype: int64


/usr/local/lib/python3.11/dist-packages/dask_expr/_groupby.py:1554: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._meta = self.obj._meta.groupby(


IDADE
1      479138
2      877931
3     1046721
4      460816
5      266852
6      178466
7      131126
8      103276
9       86265
10      71227
11     233992
12     132193
13      98386
14      70809
15      43948
16      26552
17      14996
18       6526
19       2577
20       1147
dtype: int64


/usr/local/lib/python3.11/dist-packages/dask_expr/_groupby.py:1554: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._meta = self.obj._meta.groupby(


TP_SEXO
Feminino     2624621
Masculino    1708323
dtype: int64


...

WI_FI
Não                                 533507
Sim, um ou dois dias por semana    3799437
dtype: int64


/usr/local/lib/python3.11/dist-packages/dask_expr/_groupby.py:1554: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._meta = self.obj._meta.groupby(


PC
E                                          72210
Não                                      2128452
Sim, pelo menos cinco dias por semana     162078
Sim, três ou quatro dias por semana       432995
Sim, um ou dois dias por semana          1537209
dtype: int64


/usr/local/lib/python3.11/dist-packages/dask_expr/_groupby.py:1554: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._meta = self.obj._meta.groupby(


TP_ESCOLA_EM
Não frequentei escola de Ensino Médio                                               13227
Parte em escola pública e parte em escola privada com bolsa de estudo integral      80173
Parte em escola pública e parte em escola privada sem bolsa de estudo integral     163177
Somente em escola privada com bolsa de estudo integral                             163271
Somente em escola privada sem bolsa de estudo integral                             630313
Somente em escola pública                                                         3282783
dtype: int64


In [ ]:
print('Resumo estatístico das colunas:')
df_padronizado.describe().compute()

Resumo estatístico das colunas:


,NU_INSCRICAO,NU_ANO
count,4.332944e+06,4332944.0
mean,2.100643e+11,2024.0
std,1.279403e+06,0.0
min,2.100621e+11,2024.0
25%,2.100632e+11,2024.0
50%,2.100643e+11,2024.0
75%,2.100652e+11,2024.0
max,2.100665e+11,2024.0


Escrita da base

In [ ]:
df_padronizado.to_parquet(
    'enem_1998/',
    engine='pyarrow',           # recomendado
    compression='snappy',       # rápido e eficiente
    write_index=False           # geralmente desnecessário em cargas
)

# df.to_csv("enem_1998.csv", single_file=True, index=False) #Escrita em csv

/usr/local/lib/python3.11/dist-packages/pyarrow/pandas_compat.py:625: FutureWarning: The dtype of the 'categories' of the passed categorical values (int64) does not match the specified type (string). For now ignoring the specified type, but in the future this mismatch will raise a TypeError
  arrays = [convert_column(c, f)
/usr/local/lib/python3.11/dist-packages/pyarrow/pandas_compat.py:625: FutureWarning: The dtype of the 'categories' of the passed categorical values (int64) does not match the specified type (string). For now ignoring the specified type, but in the future this mismatch will raise a TypeError
  arrays = [convert_column(c, f)
/usr/local/lib/python3.11/dist-packages/pyarrow/pandas_compat.py:625: FutureWarning: The dtype of the 'categories' of the passed categorical values (float64) does not match the specified type (string). For now ignoring the specified type, but in the future this mismatch will raise a TypeError
  arrays = [convert_column(c, f)
/usr/local/lib/python3.